# 1. RNN in PyTorch
 - input_data : 세 개의 차원을 갖는 텐서이다. (-, -, -)
 - outputs : 세 개의 차원을 갖는 텐서이다. (-, -, -)
 - PyTorch에서 RNN을 구동하기 위해서는 아래 두 줄만 적으면 된다!

```python
# RNN 모델 만들어 두기 : 셀 A를 선언하는 과정
rnn = torch.nn.RNN(input_size, hidden_size) # torch.nn.RNN에 input_size와 hidden_size를 파라미터로 입력

# RNN 모델 이용하기 : Input으로 x(t)를 넣어 셀을 거쳐 Output으로 h(t)를 구하는 과정
outputs, _status = rnn(input_data) # rnn 모델에 input_data를 넣어 outputs를 구한다.
```

# 2. RNN에서 'hello' 예제를 통해 input_data와 outputs의 디멘션 이해하기
#### (1) Input
 - 'hello'

<br>

#### (2) One-hot encoding
 - 컴퓨터가 'hello'를 이해할 수 있도록, 벡터로 치환해야 한다.
 - One-hot encoding : 단어를 구성하는 문자들을 사전식으로 나열하여, 사전의 개수만큼 벡터를 만들어 놓고 각 문자의 index에 해당하는 자리를 1로, 나머지는 0으로 구성한다.
 - ex) 
 
 'hello' 중 h는 첫 번째로 나왔으니 [1, 0, 0, 0]으로 치환한다.
 
 h를 표현하기 위해서 4개의 벡터가 필요하다는 걸 알 수 있다.
 
 즉, input으로 4개의 차원을 받아야 하기 때문에 input_size는 4가 된다.
 
 이 input_size인 4는 input_data의 3개의 디멘션 중 하나로 들어간다. (-, -, 4)

```python
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]
```

<br>

#### (3) Hidden State
 - Hidden state는 '숨겨진 상태'라는 뜻이다. 셀 A의 입장에서 'output'으로 가는 경우는 외부로 노출 되지만, 다음 셀 A로 가는 경우는 외부로 노출 되지 않는다. 따라서 Hidden state라고 한다.
 - Hidden state 에서 출력 직전에 똑같은 값이 두 개의 갈래로 갈라진다. 하나는 Outputs으로 출력 되고, 다른 하나는 Hidden state로 출력 되어 다음 셀 A로 전달 된다. (따라서 Hidden state의 size와 Outputs의 size는 같은 값을 가진다!)
 - hidden_size는 셀을 거쳐 어떤 벡터 사이즈의 출력을 원하는가를 설정한다.
 - hidden_size = 2 : 출력으로 2개의 차원을 원한다.
 - ex) 
 
 Input으로 'hello'를 받아 '반갑다' 혹은 '반갑지 않다' 2개의 감정 중 하나로 출력하고 싶은 경우 hidden_size는 2이다.
 
 이 hidden_size인 2는 outpus의 3개의 디멘션 중 하나로 들어간다. (-, -, 2)

<br>

#### (4) Sequence Length
 - Sequence Length : 데이터가 x0, x1 ... xt의 t+1개라면, t+1개의 길이를 가지는 sequence가 생긴다. 이 t+1을 sequence length라고 한다.
 - ex) 

Input으로 'hello'를 받으면, 'hello'는 5개의 문자로 이루어져 있으므로 sequence Length는 5이다.

이 Sequence Length는 사용자가 모델에게 알려주지 않아도, 모델이 자동으로 파악하는 값이다.

이 Sequence Length를 input_data의 3개의 디멘션 중 하나로 설정하면, (-, 5, 4)

outputs의 3개의 디멘션 중 하나로 자동으로 들어간다. (-, 5, 2)

```python
h = x0 = [1, 0, 0, 0]
e = x1 = [0, 1, 0, 0]
l = x2 = [0, 0, 1, 0]
l = x3 = [0, 0, 1, 0]
o = x4 = [0, 0, 0, 1]
```

<br>

#### (5) Batch Size
 - RNN에서도 여러 개의 데이터를 batch로 묶어서 모델에게 학습시킬 수 있다.
 - ex)
 
 Batch Size = 3인 경우
 
 이 Batch Size는 사용자가 모델에게 알려주지 않아도, 모델이 자동으로 파악하는 값이다.
  
 이 Batch Size를 input_data의 3개의 디멘션 중 하나로 설정하면, (3, 5, 4)
 
 outputs의 3개의 디멘션 중 하나로 자동으로 들어간다. (3, 5, 2)

```python
# Batch Size = 3인 경우
# h, e, l, o로 만들어진 수 많은 단어 중에서 3개의 단어를 묶어 배치로 구성하고, 모델에게 학습시킨다.
[h, e, l, l, o]
[e, o, l, l, l]
[l, l, e, e, l]

# 3. RNN 'hello' 예제 실습

In [1]:
# 1. 라이브러리 로드
import torch
import numpy as np

In [2]:
# 2. input_size와 hidden_size를 계산한 후 정의
input_size = 4
hidden_size = 2

In [3]:
# 3. 데이터 정제 1 : One-hot encoding
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [4]:
# 4. 데이터 정제 2 : input 데이터를 Batch Size 3으로 구성
input_data_np = np.array([[h, e, l, l, o], # Sequence Length : 5
                          [e, o, l, l, l],
                          [l, l, e, e, l]], dtype=np.float32)
# input_size의 shape은 (3, 5, 4)임을 알 수 있다.
# outputs의 shape은 (3, 5, 2)임을 알 수 있다.

In [5]:
# 5. 데이터 정제 3 : input 데이터 텐서화
input_data = torch.Tensor(input_data_np)

In [6]:
# 6. RNN 1 : 모델 만들어 두기
rnn = torch.nn.RNN(input_size, hidden_size)

In [7]:
# 7. RNN 2 : 모델 이용하기
outputs, _status = rnn(input_data)